In [151]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
import myUtils as utils
tech = "warm"  #biogas,battery, biomass, ror, geothermal，transmission
rootPath = f"../../results/scenario/{tech}"
pathTail = "/networks/base_s_27_3H_3H_2045.nc"


plt.style.use('bmh')
%matplotlib inline

nBase = pypsa.Network(f"../../results/scenario/base{pathTail}")
nMean = pypsa.Network(f"{rootPath}_mean/{pathTail}")
nGood = pypsa.Network(f"{rootPath}_good/{pathTail}")
nBest = pypsa.Network(f"{rootPath}_best/{pathTail}")

networks = {
  "nBase": nBase,
  "nMean": nMean,
  "nGood": nGood,
  "nBest": nBest
}

INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [152]:
import pandas as pd
import myUtils as utils
import importlib
importlib.reload(utils)


def getDfValue (n):
  lists = [
    n.statistics.capex().sum() / 1e9,
    n.statistics.capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.opex(groupby="de_grouper").sum() / 1e9,

    n.statistics.expanded_capex(groupby="de_all_generator_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_all_generator_grouper").sum() / 1e9,
  ]
  return lists

data = {
  'Name': [
    "CAPEX(inkl. Nachbarländer)",
    "CAPEX",
    "investierte CAPEX",
    "OPEX",

    "Strom CAPEX",
    "Strom Opex"
  ],
  # 'Unit': [
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR'
  # ],
  'Base': getDfValue(nBase),
  'Mean': getDfValue(nMean),
  'Optimist': getDfValue(nGood),
  'Best': getDfValue(nBest)
}
print(len(data['Name']), len(data['Name']), len(data['Base']))



df = pd.DataFrame(data).set_index('Name')

caepx_base = df.loc['CAPEX', 'Base']
caepx_changes = (df.loc['CAPEX', ['Mean', 'Optimist', 'Best']] - caepx_base) / caepx_base * 100

elc_caepx_base = df.loc['Strom CAPEX', 'Base']
elc_caepx_changes = (df.loc['Strom CAPEX', ['Mean', 'Optimist', 'Best']] - elc_caepx_base) / elc_caepx_base * 100

table = utils.dataframe_to_table(df)
print(table)
print(caepx_changes)
print(elc_caepx_changes)

6 6 6
  [CAPEX(inkl. Nachbarländer)], [637,328], [617,583], [614,397], [611,858],
  [CAPEX], [74,925], [69,997], [70,321], [71,383],
  [investierte CAPEX], [63,338], [58,178], [58,502], [59,564],
  [OPEX], [5,494], [5,183], [5,013], [4,958],
  [Strom CAPEX], [25,692], [20,939], [20,569], [20,415],
  [Strom Opex], [25,692], [20,939], [20,569], [20,415],
Mean       -6.57700
Optimist   -6.14524
Best       -4.72746
Name: CAPEX, dtype: float64
Mean       -18.49954
Optimist   -19.93826
Best       -20.54087
Name: Strom CAPEX, dtype: float64


In [153]:
import pandas as pd
import myUtils as utils
import importlib
importlib.reload(utils)

df = pd.DataFrame()

for key, n in networks.items():
  generator_df = utils.getNByGroup(n, 'Generator', utils.de_generator_grouper)
  generators = generator_df.groupby('carrier').sum().p_nom_opt / 1e3
  g_links_df = utils.getNByGroup(n, 'Link', utils.de_generator_grouper)
  generatorLinks = g_links_df.groupby('carrier').sum().p_nom_opt / 1e3

  h2_df = utils.getNByGroup(n, 'Link', utils.de_h2_dispatch_grouper)
  h2s =  h2_df.groupby('carrier').sum().p_nom_opt / 1e3

  biomass = utils.getNByGroup(n,'Link',utils.de_biomass_chp)
  # print(biomass)
  s = pd.concat([generators, generatorLinks,h2s])
  s['Offwind'] = s['offwind-ac'] + s['offwind-dc'] + s['offwind-float']
  s['Solar'] = s['solar'] + s['solar-hsat']
  s['Onwind'] = s['onwind']
  

  s['Solardach'] = s['solar rooftop']
  s['Erdgas CHP'] = s['urban central CHP']
  s['Geothermal'] = s.get('geothermal organic rankine cycle', 0)
  s['Biomasse CHP'] = biomass.p_nom_opt.sum() 
  s['Gesamt'] = s.sum()
  
  s = s.reindex(['Onwind', 'Offwind', 'Solar', 'Solardach','OCGT','Erdgas CHP', 'Geothermal','H2 Fuel Cell', 'H2 turbine', 'Biomasse CHP', 'ror', 'Gesamt'])
  df[key] = s

# df = df[(df != 0).any(axis=1)]
df = df.fillna(0)


print(utils.dataframe_to_table(df))

print(df)


  [Onwind], [83,101], [53,187], [53,187], [53,187],
  [Offwind], [78,329], [78,305], [78,305], [78,305],
  [Solar], [50,287], [50,287], [50,287], [50,287],
  [Solardach], [107,327], [98,194], [88,137], [80,355],
  [OCGT], [73,804], [70,483], [92,116], [111,151],
  [Erdgas CHP], [61,302], [23,198], [8,418], [0,001],
  [Geothermal], [0,000], [0,000], [0,000], [0,000],
  [H2 Fuel Cell], [0,000], [0,000], [0,000], [0,000],
  [H2 turbine], [0,000], [0,000], [0,000], [0,000],
  [Biomasse CHP], [0,000], [0,000], [0,000], [0,000],
  [ror], [0,000], [0,000], [0,000], [0,000],
  [Gesamt], [834,497], [676,826], [648,784], [635,421],
                 nBase     nMean     nGood     nBest
carrier                                             
Onwind        83.10055  53.18700  53.18701  53.18702
Offwind       78.32878  78.30491  78.30491  78.30490
Solar         50.28742  50.28731  50.28737  50.28739
Solardach    107.32740  98.19409  88.13655  80.35478
OCGT          73.80419  70.48307  92.11590 111.15131